In [1]:
import numpy as np
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm

## TF Jura table

In [2]:
tf_jura = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/Y3/desi_pv_jura_healpix.fits')
tf_jura[:5]

TARGETID,TARGET_RA,TARGET_DEC,MEAN_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_RA,STD_FIBER_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN,PVTYPE,SGA_ID,PHOTSYS,SKY_FIBER_DIST,SKY_FIBER_DIST_R26
int64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6,float64,int64,bytes3,int32,bytes1,float64,float64
-430502046,134.00013295228783,5.934552839997555,134.0000012447711,5.934402495943119,0.0,0.0,1.5211813094599733,9.536633276548175e-05,GALAXY,21.043927863240242,2049,TFT,838970,S,0.0,0.0
-427872363,61.981718642816155,-22.823913110732356,61.981815755036905,-22.824112647850505,0.01692960597574711,0.01899055577814579,0.049085960824979194,8.765385426009584e-06,GALAXY,320.327581204474,2049,TFT,982213,S,0.0,0.0
-411444222,156.2181500220905,7.1584828480845655,156.21807676457115,7.1586258676041306,0.0,0.0,0.7655265334224575,0.00014819907104570706,GALAXY,2.862433671951294,2053,TFT,4614,S,0.0,0.0
-261707523,51.18809619857989,-15.380001206793725,51.18805131712048,-15.379972719695512,0.18443052470684052,0.270201712846756,0.11706841864860068,1.263477560383397e-05,GALAXY,5022.345633155666,512,SGA,788458,S,0.0,0.0
-260779407,138.32747528464944,17.5639019859795,138.32760054746495,17.56391743552376,0.0,0.0,0.08569271543378945,9.438671035622181e-06,GALAXY,243.51359286904335,2560,TFT,735997,S,0.0,0.0


## checking if jura actually does contain iron and fuji..... hmmmm

In [3]:
tf_fuji = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/desi_pv_tf_fuji_healpix.fits')
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195


In [4]:
tf_iron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/desi_pv_tf_iron_healpix.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061


## SGA TABLE

In [5]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [6]:
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i

## calculate distances of observations

In [7]:
tf_iron['SKY_FIBER_DIST'] = np.nan
tf_iron['SKY_FIBER_DIST_R26'] = np.nan

tf_fuji['SKY_FIBER_DIST'] = np.nan
tf_fuji['SKY_FIBER_DIST_R26'] = np.nan

# For each SGA galaxy that has at least one observation, calculate the distance 
# for all of that galaxy's targets
for sga_id in tqdm(np.unique(tf_iron['SGA_ID'])):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = tf_iron['SGA_ID'] == sga_id
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center of the SGA 
    # galaxy
    #---------------------------------------------------------------------------
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tf_iron['RA'][obs_idx], 
                             dec=tf_iron['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(SGA_coords)
    
    tf_iron['SKY_FIBER_DIST'][obs_idx] = sep2d
    tf_iron['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)
    
# repeat for fuji    
for sga_id in tqdm(np.unique(tf_fuji['SGA_ID'])):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = tf_fuji['SGA_ID'] == sga_id
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center of the SGA 
    # galaxy
    #---------------------------------------------------------------------------
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tf_fuji['RA'][obs_idx], 
                             dec=tf_fuji['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(SGA_coords)
    
    tf_fuji['SKY_FIBER_DIST'][obs_idx] = sep2d
    tf_fuji['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)

100%|██████████| 4524/4524 [00:11<00:00, 395.01it/s]


In [8]:
tf_jura['SKY_FIBER_DIST'] = np.nan
tf_jura['SKY_FIBER_DIST_R26'] = np.nan

# For each SGA galaxy that has at least one observation, calculate the distance 
# for all of that galaxy's targets
for sga_id in tqdm(np.unique(tf_jura['SGA_ID'])):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = tf_jura['SGA_ID'] == sga_id
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center of the SGA 
    # galaxy
    #---------------------------------------------------------------------------
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tf_jura['MEAN_FIBER_RA'][obs_idx], 
                             dec=tf_jura['MEAN_FIBER_DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(SGA_coords)
    
    tf_jura['SKY_FIBER_DIST'][obs_idx] = sep2d
    tf_jura['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)


100%|██████████| 146579/146579 [10:47<00:00, 226.45it/s]


## calculate distances of targets

In [7]:
tf_jura['TARGET_DIST'] = np.nan
tf_jura['TARGET_DIST_R26'] = np.nan

# For each SGA galaxy that has at least one observation, calculate the distance 
# for all of that galaxy's targets
for sga_id in tqdm(np.unique(tf_jura['SGA_ID'])):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = tf_jura['SGA_ID'] == sga_id
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center of the SGA 
    # galaxy
    #---------------------------------------------------------------------------
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tf_jura['TARGET_RA'][obs_idx], 
                             dec=tf_jura['TARGET_DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(SGA_coords)
    
    tf_jura['TARGET_DIST'][obs_idx] = sep2d
    tf_jura['TARGET_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)


100%|██████████| 146579/146579 [07:32<00:00, 324.16it/s]


In [9]:
tf_master_table = vstack([tf_fuji, tf_iron, tf_jura])

## apply selection criteria
galaxies have to meet at least one of the two following criteria
1. center observation </br>
    a. 1 center observation</br>
    b. 2 non-center observations at unique distances</br>
    
2. no center observation</br>
    a. 2 observations on opposite sides of the center at the same distnace</br>
    b. 1 observation at a unique distance from the center</br>

In [22]:
#new version
tf_master_table['ROT_CURVE'] = 0.
center_dist_lim = 0.001 # target distance from center that's still considered a center observation in units of R26
unique_dist_lim = 0.01 # minimum difference for distances to be considered unique in units of R26

for sga_id in tqdm(np.unique(tf_master_table['SGA_ID'])):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = np.logical_and(tf_master_table['SGA_ID'] == sga_id, tf_master_table['TARGETID'] > 0)
    
    obs = tf_master_table[obs_idx]
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    # check that there are at least 3 observations on the galaxy
    if len(obs) >= 3:
        
        # check if galaxy has a center observation
        if np.any(obs['SKY_FIBER_DIST_R26'] < center_dist_lim):

            not_center = obs[obs['SKY_FIBER_DIST_R26'] > unique_dist_lim]
            
            if len(not_center) >= 2:
            
                #chekc that there are 2 unique distances (non-center)
                if (np.max(not_center['SKY_FIBER_DIST_R26']) - np.min(not_center['SKY_FIBER_DIST_R26'])) >= unique_dist_lim:

                    # galaxy passes criteria 1
                    tf_master_table['ROT_CURVE'][obs_idx] = 1
            


100%|██████████| 147219/147219 [11:21<00:00, 216.17it/s]


In [23]:
print('pass criteria 1: ')
print(len(tf_master_table[tf_master_table['ROT_CURVE']==1]))

pass criteria 1: 
27833


In [24]:
rot_curv_targs = tf_master_table[tf_master_table['ROT_CURVE']==1]
np.unique(rot_curv_targs['SGA_ID'])

55
117
123
146
433
793
896
937
1198
1478
1548


In [34]:
for sga_id in tqdm(np.unique(tf_master_table['SGA_ID'])):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = np.logical_and(tf_master_table['SGA_ID'] == sga_id, tf_master_table['TARGETID'] > 0)

    obs = tf_master_table[obs_idx]

    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]  
    
    if len(obs)>=3:

        if np.min(obs['SKY_FIBER_DIST_R26']) > center_dist_lim:

            # separate "left" and "right" observations
            left = obs[obs['TARGET_DEC'] - SGA['DEC'][sga_idx] > 0]
            right = obs[obs['TARGET_DEC'] - SGA['DEC'][sga_idx] < 0]

            if len(left) > 0 and len(right) > 0:

                # check that there are 2 observations at same distance on opposite sides
                for i in range(len(left)):
                    if np.any(right['SKY_FIBER_DIST_R26'] - left['SKY_FIBER_DIST_R26'][i]) < unique_dist_lim:

                        if np.abs(np.min(left['SKY_FIBER_DIST_R26']) - left['SKY_FIBER_DIST_R26'][i]) > unique_dist_lim:
                            # galaxy passes criteria 2
                            tf_master_table['ROT_CURVE'][obs_idx] = 1

                        elif np.abs(np.max(left['SKY_FIBER_DIST_R26']) - left['SKY_FIBER_DIST_R26'][i]) > unique_dist_lim:
                            # galaxy passes criteria 2
                            tf_master_table['ROT_CURVE'][obs_idx] = 1

                        elif np.abs(np.max(right['SKY_FIBER_DIST_R26']) - left['SKY_FIBER_DIST_R26'][i]) > unique_dist_lim:
                            # galaxy passes criteria 2
                            tf_master_table['ROT_CURVE'][obs_idx] = 1

                        elif np.abs(np.min(right['SKY_FIBER_DIST_R26']) - left['SKY_FIBER_DIST_R26'][i]) > unique_dist_lim:
                            # galaxy passes criteria 2
                            tf_master_table['ROT_CURVE'][obs_idx] = 1


100%|██████████| 146579/146579 [07:38<00:00, 320.01it/s]


In [ ]:
print('pass criteria 1 or 2: ')
print(len(tf_master_table[tf_master_table['ROT_CURVE']==1]))

In [25]:
# tf_jura.write('/pscratch/sd/n/nravi/pv_rot_curves/desi_pv_tf_jura_healpix_rotcurve.fits', format='fits', overwrite=True)
tf_master_table.write('/pscratch/sd/n/nravi/pv_rot_curves/desi_pv_tf_master_healpix_rotcurve.fits', format='fits', overwrite=True)